In [ ]:
from google.cloud import bigquery
import pandas as pd
import concurrent.futures
import os
import time
import logging
import datetime
import sys
import DBbtypes #코드에서 사용은 안 하지만 pip install 받아야 함.


path = "/Users/jun/GitStudy/commerceAB"
data_path = os.path.join(path, 'data')
thelook_ecommerce_path = os.path.join(data_path, "thelook_ecommerce")
os.makedirs(path, exist_ok=True)

# 로그 저장 위치
log_file = f"{path}/download_tables.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file, mode='a'),
        logging.StreamHandler()  # 콘솔 출력도 유지
    ]
)

logger = logging.getLogger("BQ_Downloader")

# BigQuery Client 생성 및 인증 테스트
try:
    client = bigquery.Client(project="promotionab")
    # 간단한 쿼리로 인증 확인
    client.query("SELECT 1").result()
    logger.info("✅ BigQuery 인증 성공")
except Exception as e:
    logger.error(f"❌ BigQuery 인증 실패: {e}")
    print(f"❌ BigQuery 인증 실패: {e}")
    sys.exit(1)  # 인증 실패 시 종료


client = bigquery.Client(project="promotionab")

tables = {
    "distribution_centers": ["id", "name", "latitude", "longitude", "distribution_center_geom"],
    "events": ["id", "user_id", "sequence_number", "session_id", "state", "city", "postal_code", "browser", "traffic_source", "uri", "event_type"],
    "inventory_items": ["id", "product_id", "created_at", "sold_at", "product_sku", "product_distribution_center_id", "product_department"],
    "order_items": ["id", "order_id", "user_id", "product_id", "inventory_item_id", "status", "created_at", "shipped_at", "delivered_at", "returned_at", "sale_price"],
    "orders": [ "order_id", "user_id", "status", "gender", "created_at", "returned_at", "shipped_at", "delivered_at", "num_of_item"],
    "products": ["id", "cost", "category", "name", "brand", "retail_price", "department", "sku", "distribution_center_id"],
    "users": ["id", "age", "gender", "state", "city", "created_at", "traffic_source"]
}

def download_table(table, cols, max_retries=3):
    """BigQuery 테이블을 parquet로 다운로드. 실패 시 재시도 (fallback)"""
    cols_str = ", ".join(cols)
    query = f"SELECT {cols_str} FROM `bigquery-public-data.thelook_ecommerce.{table}`"
    parquet_path = f"{thelook_ecommerce_path}/{table}_gzip.parquet"

    for attempt in range(1, max_retries + 1):
        try:
            print(f"[{table}] 다운로드 시도 {attempt}/{max_retries}")
            df = client.query(query).to_dataframe()
            df.to_parquet(parquet_path, index=False, compression="gzip")
            logger.info(f"✅ {table} 저장 완료 : {parquet_path}\n")
            print(f"✅ Saved: {parquet_path}\n")
            return  # 성공 시 함수 종료
        except Exception as e:
            logger.warning(f"⚠️ {table} 다운로드 실패 (시도 {attempt}): {e}")
            print(f"⚠️ {table} 실패 (시도 {attempt}): {e}")
            time.sleep(3 * attempt)  # 재시도 간 지연 (지수 백오프)
    
    # 모든 재시도 실패 시
    logger.error(f"❌ {table} 완전 실패, 로그에 기록됨.")
    with open(f"{path}/failed_tables.log", "a") as f:
        f.write(f"{datetime.now().isoformat()} - {table} failed after {max_retries} attempts\n")
    print(f"❌ {table} 완전 실패, 로그에 기록됨.\n")


# 병렬 실행
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(download_table, t, c) for t, c in tables.items()]
    concurrent.futures.wait(futures)
    executor.shutdown(wait=True)

print("✅ 모든 테이블 다운로드 완료")
sys.exit(0)


2025-10-08 22:35:37,901 - INFO - ✅ BigQuery 인증 성공


[distribution_centers] 다운로드 시도 1/3[events] 다운로드 시도 1/3
[inventory_items] 다운로드 시도 1/3

[order_items] 다운로드 시도 1/3
[orders] 다운로드 시도 1/3
[products] 다운로드 시도 1/3
[users] 다운로드 시도 1/3


/opt/homebrew/Caskroom/miniconda/base/envs/leo4study/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-10-08 22:35:41,198 - INFO - ✅ distribution_centers 저장 완료 : /Users/jun/GitStudy/commerceAB/data/distribution_centers_gzip.parquet



✅ Saved: /Users/jun/GitStudy/commerceAB/data/distribution_centers_gzip.parquet



/opt/homebrew/Caskroom/miniconda/base/envs/leo4study/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-10-08 22:35:49,278 - INFO - ✅ products 저장 완료 : /Users/jun/GitStudy/commerceAB/data/products_gzip.parquet



...

✅ Saved: /Users/jun/GitStudy/commerceAB/data/orders_gzip.parquet



2025-10-08 22:36:20,195 - INFO - ✅ order_items 저장 완료 : /Users/jun/GitStudy/commerceAB/data/order_items_gzip.parquet



✅ Saved: /Users/jun/GitStudy/commerceAB/data/order_items_gzip.parquet



2025-10-08 22:36:43,020 - INFO - ✅ inventory_items 저장 완료 : /Users/jun/GitStudy/commerceAB/data/inventory_items_gzip.parquet



✅ Saved: /Users/jun/GitStudy/commerceAB/data/inventory_items_gzip.parquet

